In [1]:
import json
import pickle
from ipymarkup import show_box_markup

import pandas as pd
from IPython.display import clear_output

from modules.utils import get_diffs

In [2]:
project_name = 'sf_upd'

In [3]:
config_path = f'out/{project_name}/config.json'

config = {}
config['df_path'] = [f'out/{project_name}/data/df/df_tsusus.pickle']
config['spans_path'] = f'out/{project_name}/data/markup/spans_true_clean_tsusus.pickle'
config['spans_pred_path'] = f'out/{project_name}/data/markup/spans_pred.pickle'


with open(config_path, 'w',  encoding='utf8') as config_file:
    json.dump(config, config_file)

In [4]:
with open(config_path,'r', encoding='utf8') as config_file:
    config = json.load(config_file)
config

{'df_path': ['out/sf_upd/data/df/df_tsusus.pickle'],
 'spans_path': 'out/sf_upd/data/markup/spans_true_clean_tsusus.pickle',
 'spans_pred_path': 'out/sf_upd/data/markup/spans_pred.pickle'}

### Чтение данных 

* df содержит название документа и его текст;
* spans_true - название документа и разметку формата (<индекс начала>, <индекс конца>, <наименование сущности>);
* spans_pred - предсказание модели

In [5]:
df = pd.DataFrame()
for path in config['df_path']:
    df_ = pd.read_pickle(path)
    df = pd.concat([df,df_])
df = df.reset_index(drop=True)
print(df.shape[0])
df.head()

602


,file,text,file_short
0,"Сч-ф_Роуминг_Сф за Февраль 2023, ка - БУ ОО МФ...",Приложение № 1 к постановлению Правительства Р...,"Сч-ф_Роуминг_Сф за Февраль 2023, ка - БУ ОО МФ..."
1,"Сч-ф_Роуминг_Сф за Февраль 2023, ка - БУ ОО МФ...",Предоставление в аренду нежилого\nпомещения,"Сч-ф_Роуминг_Сф за Февраль 2023, ка - БУ ОО МФ..."
2,УПД_Роуминг_Оплата КУ за Декабрь 2022 по дог. ...,Счет-фактура № Э/64/100157 ОТ 31 декабря 2022 ...,УПД_Роуминг_Оплата КУ за Декабрь 2022 по дог. ...
3,УПД_Роуминг_Оплата КУ за Декабрь 2022 по дог. ...,Эл. энергия по нерегулируемой цене за\n12.2022г.,УПД_Роуминг_Оплата КУ за Декабрь 2022 по дог. ...
4,"СФ_Роуминг_за Декабрь 2022, ка - БУ Сургутская...",СЧЕТ-ФАКТУРА № ООГУ-0467 от 31.12.2022 приложе...,"СФ_Роуминг_за Декабрь 2022, ка - БУ Сургутская..."


In [71]:
with open(config['spans_path'],'rb') as f:
    spans_true = pickle.load(f)
    spans_true = {k: sorted(v) for k, v in spans_true.items()}

with open(config['spans_pred_path'],'rb') as f:
    spans_pred = pickle.load(f)
    spans_pred = {k: sorted(v) for k, v in spans_pred.items()}

In [80]:
spans_true_diff, spans_pred_diff = get_diffs(spans_true, spans_pred, df)

union_keys = spans_pred_diff.keys() | spans_true_diff.keys()

len(spans_true_diff), len(spans_pred_diff), len(union_keys), len(spans_true)

(374, 207, 374, 602)

In [73]:
#u2 = pd.Series(list(union_keys), index=range(1, 402))

#with open('union_keys_todo.pickle', 'wb') as f:
#    pickle.dump(u2, f)

In [74]:
with open('union_keys_todo.pickle','rb') as f:
    u2 = pickle.load(f)

with open('union_keys_all.pickle','rb') as f:
    union_keys = pickle.load(f)

num = u2.index[0]

diff_files = set(list(df[df.file.isin(u2)]['file']))

categories = {'1': 'ADDRESSBUYER',
              '2': 'ADDRESSSELLER',
              '3': 'ADDRESSSHIPPER',
              '4': 'ADDROBJ',
              '5': 'BPINN',
              '6': 'BPKPP',
              '7': 'BPNAME',
              '8': 'CONTENT',
              '9': 'CURRENCY',
              '10': 'CURRENCYKOD',
              '11': 'DOCDATE',
              '12': 'DOGDATE',
              '13': 'DOGNUM',
              '14': 'INNBUYER',
              '15': 'KPPBUYER',
              '16': 'NAMEBUYER',
              '17': 'PERIOD',
              '18': 'SFNUM',
              '19': 'SHIPPERNAME',
              '20': 'SUMM'}

idx = iter(u2.index)
print(len(u2))

3


In [75]:
def save_data(key, spans_true, spans_add, spans_del):
    for key in spans_add.keys():
        if key in spans_true.keys():
            spans_true[key] = spans_add[key] + spans_true[key]
        else:
            spans_true[key] = spans_add[key]
            
    for key in spans_del.keys():
        if key in spans_true.keys():
            spans_true[key] = [x for x in spans_true[key] if x not in spans_del[key]]

    with open(config['spans_path'], 'wb') as f:
        pickle.dump(spans_true, f)

In [76]:
def show_info(df, key, spans_pred_diff, spans_true_diff, spans_true):
    text = df[df.file == key]['text'].values[0]
    text = text.replace('\n', ' ')

    print('Разметка модели целиком: ')
    show_box_markup(text, spans_pred[key])

    """
    print('Разметка человека: ')
    spans = spans_true_diff.get(key, [])
    print(spans)
    show_box_markup(text, spans)
    """

    print('Разметка человека целиком: ')
    show_box_markup(text, spans_true[key])

    print('Исходный текст')
    show_box_markup(text, [])

    return text

In [77]:
def find_all_string_idx(text, s):
    length = len(s)
    count = 1
    string_idx_in_text = {}
    start_index_text = 0
    while text[start_index_text:].find(s) != -1:
        start_index = text[start_index_text:].find(s) + start_index_text
        string_idx_in_text[f'{count}'] = start_index
        count += 1
        start_index_text = start_index + length
    return string_idx_in_text

#### Проверка

##### spans_true

In [85]:
# key = 'УПД_Роуминг_Оплата ЭУ за Июнь 2022 по дог. 50003889985 от 20.10.2014 № 1_page000_text.html'
union_keys = list(union_keys)
key = union_keys[290]
text = df[df.file == key]['text'].values[0]
text = text.replace('\n', ' ')
show_box_markup(text, spans_true[key])

##### spans_pred

In [27]:
key = 'Роуминг_оплата аренды с 20.07.2022г. по 31.07.2022г дог.50004220061 от 20.07. (2)_page000_text.html'
text = df[df.file == key]['text'].values[0]
text = text.replace('\n', ' ')
show_box_markup(text, spans_pred[key])

In [70]:
spans_add = {}
spans_del = {}

while True:

    num = u2.index[0]
    key = u2[num]

    new_spans = []
    del_spans = []
    count = 0

    print(f'Документ: {num}/401\n')
    print(f'Ключ: {key}\n')

    text = show_info(df, key, spans_pred_diff, spans_true_diff, spans_true)

    while True:
        var = input('1 - ДОБАВИТЬ МЕТКУ, 2 - УДАЛИТЬ, 3 - ВЫБРАТЬ ФАЙЛ') or 'blank'
        if var != 'blank':
            if var == '1':
                while True:
                    s = input('ВВЕДИТЕ ТЕКСТ РАЗМЕТКИ или ENTER: ') or 'blank'
                    if s != 'blank':
                        if text.find(s) != -1:
                            length = len(s)
                            if len(find_all_string_idx(text, s)) == 1:
                                start_index = text.find(s)
                                category = input(f'ВЫБЕРИТЕ КАТРЕГОРИЮ:\n {categories}')
                                new_spans.append((start_index, start_index+length, categories[category]))
                                var_ = input('Продолжить добавление? 1 - ДА, 2 - НЕТ')
                                if var_ == '1':
                                    continue
                                else:
                                    spans_add[key] = new_spans
                                    print(f'Итоговый текст {count}')
                                    show_box_markup(text, [x for x in new_spans+spans_true[key] if x not in del_spans])
                                    count += 1
                                    break
                            else:
                                string_idx_in_text = find_all_string_idx(text, s) 
                                idx_ = input(f'Выберите нужную строку: {string_idx_in_text}') or 'blank'
                                if idx_ != 'blank':
                                    try:
                                        start_index = string_idx_in_text[idx_]
                                        category = input(f'ВЫБЕРИТЕ КАТРЕГОРИЮ:\n {categories}')
                                        new_spans.append((start_index, start_index+length, categories[category]))
                                        var_ = input('Продолжить добавление? 1 - ДА, 2 - НЕТ')
                                    except:
                                        print('Указанной строки нет в списке')
                                    if var_ == '1':
                                        continue
                                    else:
                                        spans_add[key] = new_spans
                                        print(f'Итоговый текст {count}')
                                        show_box_markup(text, [x for x in new_spans+spans_true[key] if x not in del_spans])
                                        count += 1
                                        break
                                else:
                                    pass
                        else:
                            var_ = input('Текст не найден. Повторить попытку? 1 - ДА, ОСТАЛЬНОЕ - НЕТ')
                            if var_ == '1':
                                continue
                            else: break
                    else: break
            elif var == '2':
                while True:
                    actual_spans_list = [x for x in new_spans+spans_true[key] if x not in del_spans]
                    print(f' Актуально: {actual_spans_list}')
                    del_span_index = input(f'Выберите метку для удаления или ENTER: \
                                            {[i for i in enumerate(actual_spans_list)]}') or 'blank'
                    if del_span_index != 'blank':
                        if del_span_index.isdigit():
                            del_spans.append(actual_spans_list[int(del_span_index)])
                            var_ = input('Продолжить удаление? 1 - ДА, 2 - НЕТ')
                            if var_ == '1':
                                continue
                            else:
                                spans_del[key] = del_spans
                                print(f'Итоговый текст {count}')
                                show_box_markup(text, [x for x in new_spans+spans_true[key] if x not in del_spans])
                                count += 1
                                break
                        else:
                            print('Ошибка ввода!')
                            continue
                    else: break
            elif var == '3':
                while True:
                    num_ = input('Введите номер ключа или ENTER, чтобы вернуть исходный: ') or 'blank'
                    if num_ != 'blank':
                        if num_.isdigit():
                            try:
                                key = union_keys[int(num_)]
                                clear_output()
                                print(f'Документ изменен на: {num_}/401\n')
                                print(f'Ключ: {key}\n')
                                text = show_info(df, key, spans_pred_diff, spans_true_diff, spans_true)
                                break
                            except:
                                print('Ошибка ввода! Указанного индекса нет в файле')
                        else:
                            print('Ошибка ввода! Вы ввели не число')
                    else:
                        key = u2[num]
                        print(f'Документ: {num}/401 (исходный)\n')
                        break
        else:
            if key == u2[num]: break
            else:
                var_ = input(f'Редактирование документа {num_} завершено! 1 - Вернуться к разметке документа {num}, ENTER - завершить') or 'blank'
                if var_ == '1':
                    save_data(key, spans_true, spans_add, spans_del) # не проверено
                    spans_add = {} # не проверено
                    spans_del = {} # не проверено
                    key == u2[num]
                    clear_output()
                    print(f'Документ изменен на исходный: {num}/401\n')
                    print(f'Ключ: {key}\n')
                    text = show_info(df, key, spans_pred_diff, spans_true_diff, spans_true)
                else: break

    # сохранение данных
    save_data(key, spans_true, spans_add, spans_del)

    spans_add = {}
    spans_del = {}
    
    _var = input('ПРОДОЛЖИТЬ - 1, ЗАВЕРШИТЬ РАБОТУ - ENTER')
    if _var == '1':
        u2 = u2[1:]
        clear_output()
    else:
        var_ = input('СОХРАНИТЬ ИНДЕКС НАЧАЛА - ENTER, 1 - ПРОПУСТИТЬ')
        if var_ != '1':
            with open('union_keys_todo.pickle', 'wb') as f:
                pickle.dump(u2, f)
        break

IndexError: index 0 is out of bounds for axis 0 with size 0